In [199]:
import numpy as np
import matplotlib.pyplot as plt
import random as rand
import sklearn.svm as svm
import math

In [76]:
def point_generator(num):
    points = []
    for i in range(num):
        x_coord = rand.uniform(-1, 1)
        y_coord = rand.uniform(-1, 1)
        point = (1, x_coord, y_coord)
        points.append(point)
    return points

In [77]:
def rand_line_maker():
    point1 = (rand.uniform(-1, 1), rand.uniform(-1, 1))
    point2 = (rand.uniform(-1, 1), rand.uniform(-1, 1))
    m = (point2[1] - point1[1]) / (point2[0] - point1[0])
    intercept = -m * point1[0] + point1[1]
    return (m, intercept)

In [80]:
def classifier(points, line):
    label = 0
    training_set = []
    for i in range(len(points)):
        if points[i][1] > points[i][0] * line[0] + line[1]:
            label = 1
        else:
            label = -1
        training_set.append((points[i][0], points[i][1], points[i][2], label))
    return training_set

In [160]:
def perceptron(num):
    y = []
    points = point_generator(num)
    target_function = rand_line_maker()
    labeled_set = classifier(points, target_function)
    for i in range(num):
        y.append(labeled_set[i][3])
    while y == [1] * num or y == [-1] * num:
        y = []
        points = point_generator(num)
        target_function = rand_line_maker()
        labeled_set = classifier(points, target_function)
        for i in range(num):
            y.append(labeled_set[i][3])
    weight_vector = [0, 0, 0]
    misclassified = labeled_set
    iterations = 0
    while len(misclassified) > 0:
        mpoint = rand.choice(misclassified)
        label = mpoint[3]
        for i in range(3):
            weight_vector[i] += mpoint[i] * label
        misclassified = []
        for i in range(len(labeled_set)):
            if np.sign(weight_vector[0] * labeled_set[i][0] +
                    weight_vector[1] * labeled_set[i][1] +
                    weight_vector[2] * labeled_set[i][2]) != labeled_set[i][3]:
                misclassified.append(labeled_set[i])
    return (weight_vector, points, target_function, y)

In [5]:
in_data = np.loadtxt(fname = "C:\\Users\\Spencer\\Downloads\\in.dta.txt")

In [6]:
out_data = np.loadtxt(fname = "C:\\Users\\Spencer\\Downloads\\out.dta.txt")

In [7]:
def nonlinear_transform(pts, k):
    transformation = []
    for pt in pts:
        x1 = pt[0]
        x2 = pt[1]
        trans = [1, x1, x2, x1 ** 2, x2 ** 2, x1 * x2, abs((x1 - x2)), abs((x1 + x2))]
        transformation.append(trans[:(k+1)])
    return transformation

In [8]:
def nonlinear_transformation_regression(training_pts, labels, k):
    missclassified = 0
    transformed_pts = np.asarray(nonlinear_transform(training_pts, k))
    pseudo_inverse = np.linalg.pinv(transformed_pts)
    y = np.asarray(labels)
    w = np.matmul(pseudo_inverse, y)
    for i in range(len(training_pts)):
        if np.sign(w.dot(transformed_pts[i])) != labels[i]:
            missclassified += 1
    return (w, missclassified / len(training_pts)) 

In [133]:
def error(w, pts, labels, k):
    e_out = 0
    missclassified = 0
    transformed = np.asarray(nonlinear_transform(pts, k))
    for i in range(len(pts)):
        if np.sign(w.dot(transformed[i])) != labels[i]:
            missclassified += 1
    e_out = missclassified / len(pts)
    return e_out

In [141]:
def error_perceptron(w, pts, labels):
    e_out = 0
    missclassified = 0
    w = np.asarray(w)
    pts = np.asarray(pts)
    for i in range(len(pts)):
        if np.sign(w.dot(pts[i])) != labels[i]:
            missclassified += 1
    e_out = missclassified / len(pts)
    return e_out

In [19]:
def line_maker(a, b):
    m = (b[1] - a[1]) / (b[0] - a[0])
    intercept = -m * a[0] + a[1]
    return (m, intercept)

In [11]:
in_pts = []
for i in range(len(in_data)):
    in_pts.append((in_data[i][0], in_data[i][1]))
    
in_y = []
for i in range(len(in_data)):
    in_y.append(in_data[i][2])
    
out_pts = []
for i in range(len(out_data)):
    out_pts.append((out_data[i][0], out_data[i][1]))
    
out_y = []
for i in range(len(out_data)):
    out_y.append(out_data[i][2])

Problems 1, 2 and 5

In [134]:
errors1 = []
e_out1 = []
for i in range(5):
    i += 3
    hypo = nonlinear_transformation_regression(in_data[:25], in_y[:25], i)
    errors1.append(error(hypo[0], in_data[25:], in_y[25:], i))
    e_out1.append(error(hypo[0], out_data, out_y, i))
(errors1, e_out1)

([0.3, 0.5, 0.2, 0.0, 0.1], [0.42, 0.416, 0.188, 0.084, 0.072])

Problems 3, 4, and 5

In [13]:
errors3 = []
e_out3 = []
for i in range(5):
    i += 3
    hypo = nonlinear_transformation_regression(in_data[25:], in_y[25:], i)
    errors3.append(error(hypo[0], in_data[:25], in_y[:25], i))
    e_out3.append(error(hypo[0], out_data, out_y, i))
(errors3, e_out3)

([0.28, 0.36, 0.2, 0.08, 0.12], [0.396, 0.388, 0.284, 0.192, 0.196])

Problem 6

In [14]:
exp_value = 0
for i in range(10000):
    a = rand.uniform(0, 1)
    b = rand.uniform(0, 1)
    exp_value += min(a, b)
exp_value / 10000

0.334169321774592

Problem 8

In [205]:
better = 0
for i in range(1000):
    y = []
    x1 = perceptron(10)
    x2 = svm.SVC(C=math.inf, kernel='linear')
    x2.fit(x1[1], x1[3])
    pts = point_generator(1000)
    labels = classifier(pts, x1[2])
    for i in range(len(pts)):
            y.append(labels[i][3])
    svm_e = abs(x2.score(pts, y)-1)
    if error_perceptron(x1[0], pts, y) > svm_e:
        better += 1
better / 1000

0.585

Problems 9 and 10

In [204]:
better = 0
n_sv = 0
for i in range(1000):
    y = []
    x1 = perceptron(100)
    x2 = svm.SVC(C=math.inf, kernel='linear')
    x2.fit(x1[1], x1[3])
    pts = point_generator(10000)
    labels = classifier(pts, x1[2])
    for i in range(len(pts)):
            y.append(labels[i][3])
    svm_e = abs(x2.score(pts, y)-1)
    if error_perceptron(x1[0], pts, y) > svm_e:
        better += 1
    n_sv += len(x2.support_vectors_)
(better / 1000, n_sv / 1000)

(0.654, 2.999)